In [113]:
import pymysql.cursors
from sqlalchemy import create_engine, MetaData
from sqlalchemy.schema import CreateTable
from sqlalchemy import text
import getpass

In [114]:
#prompt user to enter MySQL root password
sql_pass = getpass.getpass()
#create connection string and engine to connect to MySQL database
connection_string = 'mysql+pymysql://root:' + sql_pass + '@localhost:3306/civictech'
engine = create_engine(connection_string)

In [115]:
import pandas as pd

In [116]:
people=pd.read_csv('profiles/profile_data_reprocessed.csv', sep=';').reset_index()
people.head()

,index,profile_id,lastName,firstName,geoCountryName,geoLocationName,birthDate,summary,industryName,headline,...,ed3_description,ed3_activities,ed3_courses,ed3_honors,ed1_startDate_month,ed1_startDate_year,ed2_startDate_month,ed2_startDate_year,ed3_startDate_month,ed3_startDate_year
0,0,ACoAAAA615EBaNquQR5gOz_oFr9emeCr0ZNw67M,Brachet,Antoine,France,Greater Paris Metropolitan Region,NaN,Antoine croit à l’intelligence de tous et à la...,IT Services and IT Consulting,"Directeur associé de bluenove, initiateur du m...",...,NaN,NaN,NaN,NaN,NaN,1998.0,NaN,1996.0,NaN,1993.0
1,1,ACoAAAfK9YwBjZr16cDaVuxZICOg0QUnbPiUoXE,Demri,Bobby,France,"Paris, Île-de-France",NaN,French Entrepreneur - Founder and Managing Par...,Venture Capital and Private Equity Principals,Founder & Managing Partner at ROCH Ventures,...,NaN,NaN,NaN,NaN,NaN,2006.0,NaN,2005.0,NaN,1998.0
2,2,ACoAAAUn_5ABqO0mSShQxo4gFyTaCDoaYUk5Fm8,Durieux,Sarah,France,"Paris, Île-de-France",NaN,Inspired by the power we can build to change t...,Civic and Social Organizations,Co-director Multitudes Foundation - Activist a...,...,Public narrative is a leadership-development p...,NaN,NaN,NaN,3.0,2022.0,9.0,2020.0,9.0,2020.0
3,3,ACoAAARi9fcBfH1GjbFHnWc1QqHftfh3LQjRUEg,Jaillot,Bastien,France,Greater Paris Metropolitan Region,NaN,"IT Consultant, Technical Expert, Lead develope...",IT Services and IT Consulting,Web expert chez Jolicode,...,NaN,NaN,NaN,NaN,NaN,2004.0,NaN,NaN,NaN,NaN
4,4,ACoAAApOhv4B3_GF-OGg8-ipxxkMf6AOggWigl8,Vanneroy,Coline,France,Greater Paris Metropolitan Region,NaN,NaN,IT Services and IT Consulting,Directrice des opérations chez Cap Collectif,...,NaN,NaN,NaN,NaN,NaN,2005.0,NaN,2002.0,NaN,NaN


In [117]:
names=people[["index", "lastName", 'firstName']]
names.columns=["id", "lastName", 'firstName']

In [118]:
names

,id,lastName,firstName
0,0,Brachet,Antoine
1,1,Demri,Bobby
2,2,Durieux,Sarah
3,3,Jaillot,Bastien
4,4,Vanneroy,Coline
5,5,DEMBELE,Aminata
6,6,de Pimodan,Julie
7,7,Rosado,Alice
8,8,Barbier,Maxime
9,9,Pahud,Chloe


In [119]:
names.to_sql('names', engine, 'civictech', if_exists='replace', index=False)

47

In [120]:
people.columns

Index(['index', 'profile_id', 'lastName', 'firstName', 'geoCountryName',
       'geoLocationName', 'birthDate', 'summary', 'industryName', 'headline',
       ...
       'ed3_description', 'ed3_activities', 'ed3_courses', 'ed3_honors',
       'ed1_startDate_month', 'ed1_startDate_year', 'ed2_startDate_month',
       'ed2_startDate_year', 'ed3_startDate_month', 'ed3_startDate_year'],
      dtype='object', length=151)

In [121]:
people.rename(columns={"index": "ID"}, inplace=True)

In [122]:
people.drop(columns=["profile_id", "lastName", 'firstName', 'experience1', 'experience2', 'experience3', 
                     'experience4', 'experience5', 'education1', 'education2', 'education3',
                     'exp1_honors', 'exp2_honors', 'exp5_honors', 
                     'ed1_school','ed2_school','ed3_school', 'ed1_activities', 'ed2_activities', 'ed3_activities', 
                     'ed1_description', 'ed2_description', 'ed3_description', 'ed1_courses', 'ed3_courses',
                     'ed1_honors', 'ed2_honors', 'ed3_honors', 'ed1_grade', 'ed2_grade', 'ed3_grade'], inplace=True)

In [123]:
#people.drop(columns=['exp3_honors', 'exp4_honors', 'ed2_courses'])

In [124]:
people.columns

Index(['ID', 'geoCountryName', 'geoLocationName', 'birthDate', 'summary',
       'industryName', 'headline', 'experience', 'education', 'languages',
       ...
       'ed3_timePeriod', 'ed3_degreeName', 'ed3_schoolName',
       'ed3_fieldOfStudy', 'ed1_startDate_month', 'ed1_startDate_year',
       'ed2_startDate_month', 'ed2_startDate_year', 'ed3_startDate_month',
       'ed3_startDate_year'],
      dtype='object', length=120)

In [125]:
people.to_sql('people', engine, 'civictech', if_exists='replace', index=False)

47

## Companies finance data

In [126]:
companies_finance=pd.read_csv('scraping_organizations/companies_finance_data.csv', sep=';') # possible index drop
print(companies_finance.columns)

Index(['Unnamed: 0', 'Chiffre d'affaires (€)', 'Marge brute (€)',
       'EBITDA - EBE (€)', 'Résultat d'exploitation (€)', 'Résultat net (€)',
       'Croissance', 'Taux de croissance du CA (%)', 'Taux de marge brute (%)',
       'Taux de marge d'EBITDA (%)', 'Taux de marge opérationnelle (%)',
       'Gestion BFR', 'BFR (€)', 'BFR exploitation (€)',
       'BFR hors exploitation (€)', 'BFR (j de CA)',
       'BFR exploitation (j de CA)', 'BFR hors exploitation (j de CA)',
       'Délai de paiement clients (j)', 'Délai de paiement fournisseurs (j)',
       'Ratio des stocks / CA (j)', 'Autonomie financière',
       'Capacité d'autofinancement (€)', 'Capacité d'autofinancement / CA (%)',
       'Fonds de roulement net global (€)', 'Couverture du BFR',
       'Trésorerie (€)', 'Dettes financières (€)', 'Capacité de remboursement',
       'Ratio d'endettement (Gearing)', 'Autonomie financière (%)',
       'Taux de levier (DFN/EBITDA)', 'Solvabilité',
       'Etat des dettes à 1 an au plu

In [127]:
import inflection
import re
new_col_names=[]
for i in list(companies_finance.columns):
    i=i.lower().replace(' de', '_').replace(' du', '_').replace("'", '').replace(' ', '_').replace('é', 'e').replace('(€)','e').replace('(%)','pc').replace('/','sur').replace('à', 'a').replace('è', 'e').replace('(', '').replace(')', '').replace('ô', 'o').replace('__', '_').replace('s_t', 'surdet')
    new_col_names.append(inflection.underscore(i))
print(new_col_names)

['unnamed:_0', 'chiffre_daffaires_e', 'marge_brute_e', 'ebitda___ebe_e', 'resultat_dexploitation_e', 'resultat_net_e', 'croissance', 'taux_croissance_ca_pc', 'taux_marge_brute_pc', 'taux_marge_debitda_pc', 'taux_marge_operationnelle_pc', 'gestion_bfr', 'bfr_e', 'bfr_exploitation_e', 'bfr_hors_exploitation_e', 'bfr_j_ca', 'bfr_exploitation_j_ca', 'bfr_hors_exploitation_j_ca', 'delai_paiement_clients_j', 'delai_paiement_fournisseurs_j', 'ratio_s_stocks_sur_ca_j', 'autonomie_financiere', 'capacite_dautofinancement_e', 'capacite_dautofinancement_sur_ca_pc', 'fonds_roulement_net_global_e', 'couverture_bfr', 'tresorerie_e', 'dettes_financieres_e', 'capacite_remboursement', 'ratio_dendettement_gearing', 'autonomie_financiere_pc', 'taux_levier_dfnsurebitda', 'solvabilite', 'etat_surdettes_a_1_an_au_plus_e', 'liquidite_generale', 'couverture_surdettes', 'fonds_propres_e', 'rentabilite', 'marge_nette_pc', 'rentabilite_sur_fonds_propres_pc', 'rentabilite_economique_pc', 'valeur_ajoutee_e', 'valeu

In [128]:
companies_finance.columns = new_col_names

In [129]:
print(list(companies_finance.columns))

['unnamed:_0', 'chiffre_daffaires_e', 'marge_brute_e', 'ebitda___ebe_e', 'resultat_dexploitation_e', 'resultat_net_e', 'croissance', 'taux_croissance_ca_pc', 'taux_marge_brute_pc', 'taux_marge_debitda_pc', 'taux_marge_operationnelle_pc', 'gestion_bfr', 'bfr_e', 'bfr_exploitation_e', 'bfr_hors_exploitation_e', 'bfr_j_ca', 'bfr_exploitation_j_ca', 'bfr_hors_exploitation_j_ca', 'delai_paiement_clients_j', 'delai_paiement_fournisseurs_j', 'ratio_s_stocks_sur_ca_j', 'autonomie_financiere', 'capacite_dautofinancement_e', 'capacite_dautofinancement_sur_ca_pc', 'fonds_roulement_net_global_e', 'couverture_bfr', 'tresorerie_e', 'dettes_financieres_e', 'capacite_remboursement', 'ratio_dendettement_gearing', 'autonomie_financiere_pc', 'taux_levier_dfnsurebitda', 'solvabilite', 'etat_surdettes_a_1_an_au_plus_e', 'liquidite_generale', 'couverture_surdettes', 'fonds_propres_e', 'rentabilite', 'marge_nette_pc', 'rentabilite_sur_fonds_propres_pc', 'rentabilite_economique_pc', 'valeur_ajoutee_e', 'valeu

In [130]:
import re

def fix_columns(x):
    x=str(x)
    if x== 'nan':
        return 0
    elif 'K' in x:
        if ',' in x:
            return int(re.split('[,K]', x)[0]+re.split('[,K]', x)[1]+'0'*(3-len(re.split('[,K]', x)[1])))
        else:
            return int(x.replace('K', '000'))
    elif 'M' in x:
        if ',' in x:
            return int(re.split('[,M]', x)[0]+re.split('[,M]', x)[1]+'0'*(6-len(re.split('[,M]', x)[1])))
        else:
            return int(x.replace('M', '000'))
    else:
        return x

In [131]:
# to test the function on one series 
# companies_finance['chiffre_daffaires_e'].apply(fix_columns)

In [132]:
for i in list(companies_finance.columns)[1:]: 
    companies_finance[i]= companies_finance[i].apply(fix_columns)

In [133]:
companies_finance.head()

,unnamed:_0,chiffre_daffaires_e,marge_brute_e,ebitda___ebe_e,resultat_dexploitation_e,resultat_net_e,croissance,taux_croissance_ca_pc,taux_marge_brute_pc,taux_marge_debitda_pc,...,marge_nette_pc,rentabilite_sur_fonds_propres_pc,rentabilite_economique_pc,valeur_ajoutee_e,valeur_ajoutee_sur_ca_pc,structure_dactivite,salaires_et_charges_sociales_e,salaires_sur_ca_pc,impots_et_taxes_e,chiffre_daffaires_a_lexport_e
0,2017_citility,0,0,0,0,-562000,2017.0,0,0,0,...,0,0,0,0,0,2017.0,0,0,0,0
1,2016_citility,30700,527000,-295000,-296000,-238000,2016.0,4,1720,-961,...,-775,-623,-173,351000,1140,2016.0,636000,2070,9160,0
2,2019_voxcracy,46300,46300,-31500,-38500,-39000,2019.0,177,100,-682,...,-842,-76,-63,2170,47,2019.0,33600,725,137,0
3,2018_voxcracy,16700,157000,-123000,-130000,-112000,2018.0,861,940,-734,...,-668,-202,-169,-21000,-126,2018.0,101000,602,1120,5000
4,2017_voxcracy,1740,75900,-24400,-30300,-23400,2017.0,0,4360,-1400,...,-1350,-41,-39,-5040,-290,2017.0,19100,1100,329,0


In [134]:
companies_finance.dtypes

unnamed:_0                             object
chiffre_daffaires_e                    object
marge_brute_e                          object
ebitda___ebe_e                         object
resultat_dexploitation_e               object
resultat_net_e                         object
croissance                             object
taux_croissance_ca_pc                  object
taux_marge_brute_pc                    object
taux_marge_debitda_pc                  object
taux_marge_operationnelle_pc           object
gestion_bfr                            object
bfr_e                                  object
bfr_exploitation_e                     object
bfr_hors_exploitation_e                object
bfr_j_ca                               object
bfr_exploitation_j_ca                  object
bfr_hors_exploitation_j_ca             object
delai_paiement_clients_j               object
delai_paiement_fournisseurs_j          object
ratio_s_stocks_sur_ca_j                object
autonomie_financiere              

In [135]:
## change data types 
companies_finance.set_index("unnamed:_0", drop=True, inplace=True)

In [136]:
companies_finance=companies_finance.apply(pd.to_numeric).reset_index()

In [137]:
companies_finance.dtypes
companies_finance.head()

,unnamed:_0,chiffre_daffaires_e,marge_brute_e,ebitda___ebe_e,resultat_dexploitation_e,resultat_net_e,croissance,taux_croissance_ca_pc,taux_marge_brute_pc,taux_marge_debitda_pc,...,marge_nette_pc,rentabilite_sur_fonds_propres_pc,rentabilite_economique_pc,valeur_ajoutee_e,valeur_ajoutee_sur_ca_pc,structure_dactivite,salaires_et_charges_sociales_e,salaires_sur_ca_pc,impots_et_taxes_e,chiffre_daffaires_a_lexport_e
0,2017_citility,0.0,0,0,0,-562000,2017.0,0,0,0,...,0,0,0,0,0,2017.0,0,0,0,0
1,2016_citility,30700.0,527000,-295000,-296000,-238000,2016.0,4,1720,-961,...,-775,-623,-173,351000,1140,2016.0,636000,2070,9160,0
2,2019_voxcracy,46300.0,46300,-31500,-38500,-39000,2019.0,177,100,-682,...,-842,-76,-63,2170,47,2019.0,33600,725,137,0
3,2018_voxcracy,16700.0,157000,-123000,-130000,-112000,2018.0,861,940,-734,...,-668,-202,-169,-21000,-126,2018.0,101000,602,1120,5000
4,2017_voxcracy,1740.0,75900,-24400,-30300,-23400,2017.0,0,4360,-1400,...,-1350,-41,-39,-5040,-290,2017.0,19100,1100,329,0


In [138]:
companies_finance.to_sql('companies_finance', engine, 'civictech', if_exists='replace', index=False)

74

In [139]:
companies_info=pd.read_csv('scraping_organizations/companies_info_data.csv', sep=';') # possible index drop
print(companies_info.columns)

Index(['Unnamed: 0', 'Adresse :', 'Activité :', 'Effectif :', 'Création :',
       'Dirigeants :', 'SIREN :', 'SIRET (siège) :', 'Forme juridique :',
       'Numéro de TVA:', 'Inscription au RCS :', 'Numéro RCS :',
       'Capital social :', 'Activité principale déclarée :',
       'Code NAF ou APE :', 'Domaine d’activité :', 'Dirigeant :',
       'Capital variable (minimum) :', 'Convention collective  :',
       'Inscription au RNA :', 'Identifiant association :',
       'Objet de l'association :', 'Statut INSEE :'],
      dtype='object')


In [140]:
new_cols=[]
for i in list(companies_info.columns):
    i=i.lower().replace(' :', '').replace('é', 'e').replace("(", '').replace(')', '').replace(' ', '_').replace("'",'').replace("’",'').replace("è", "e")
    new_cols.append(i)
print(new_cols)

['unnamed:_0', 'adresse', 'activite', 'effectif', 'creation', 'dirigeants', 'siren', 'siret_siege', 'forme_juridique', 'numero_de_tva:', 'inscription_au_rcs', 'numero_rcs', 'capital_social', 'activite_principale_declaree', 'code_naf_ou_ape', 'domaine_dactivite', 'dirigeant', 'capital_variable_minimum', 'convention_collective_', 'inscription_au_rna', 'identifiant_association', 'objet_de_lassociation', 'statut_insee']


In [141]:
companies_info.columns=new_cols
companies_info.head()

,unnamed:_0,adresse,activite,effectif,creation,dirigeants,siren,siret_siege,forme_juridique,numero_de_tva:,...,activite_principale_declaree,code_naf_ou_ape,domaine_dactivite,dirigeant,capital_variable_minimum,convention_collective_,inscription_au_rna,identifiant_association,objet_de_lassociation,statut_insee
0,citility,5 RUE DE LA CLAIRE 69009 LYON 9EME,Édition de logiciels applicatifs,0 salarié (donnée 2019),05/05/2014,"André MAY, ODICEO, Sabine SCHNECK",802 503 276,802 503 276 00023,"SAS, société par actions simplifiée",FR07802503276,...,Edition de logiciels applicatifs.,58.29C (Édition de logiciels applicatifs),Édition,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,poligma,RPT BENJAMIN FRANKLIN 34960 MONTPELLIER CEDEX 2,Conseil en systèmes et logiciels informatiques,Entre 3 et 5 salariés (donnée 2020),01/09/2015,NaN,813 463 601,813 463 601 00027,"SAS, société par actions simplifiée",FR19813463601,...,Développement de services informatiques et num...,62.02A (Conseil en systèmes et logiciels infor...,"Programmation, conseil et autres activités inf...",Philippe GERARD,NaN,NaN,NaN,NaN,NaN,NaN
2,voxcracy,1133 RTE FENERIE 06580 PEGOMAS,Programmation informatique,0 salarié,01/09/2014,"Olivier ROCCA, Pascal RUSCICA",803 548 031,803 548 031 00019,"SAS, société par actions simplifiée",FR08803548031,...,"La recherche, le développement et la commercia...",62.01Z (Programmation informatique),"Programmation, conseil et autres activités inf...",NaN,"100,00 €",NaN,NaN,NaN,NaN,NaN
3,LLL_2,16 RUE DU CAIRE 75002 PARIS 2,Autres activités de soutien aux entreprises n....,Entre 3 et 5 salariés (donnée 2020),16/01/2015,"ANNAMAMASHOW, Raymond Maeder, 3APEXCO",809 375 744,809 375 744 00012,"SAS, société par actions simplifiée",FR61809375744,...,Développement de nouvelles formes de collabora...,82.99Z (Autres activités de soutien aux entrep...,Activités administratives et autres activités ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bluenove,112 B RUE CARDINET 75017 PARIS 17,Conseil pour les affaires et autres conseils d...,Entre 20 et 49 salariés (donnée 2020),23/01/2008,"GROUPE BLUENOVE INC., Guillaume Drancy, Carole...",502 204 944,502 204 944 00057,"SAS, société par actions simplifiée",FR77502204944,...,L'activité de conseil en stratégie d'accompagn...,70.22Z (Conseil pour les affaires et autres co...,Activités des sièges sociaux ; conseil de gestion,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
companies_info.to_sql('companies_info', engine, 'civictech', if_exists='replace', index=False)

76